In [2]:
import os
import zipfile
import pandas as pd

In [5]:
schleswig_holstein_weather_station_ids = ['05516', '02564', '04466', '03032', '02115']

In [10]:
# Extract schleswig holstein wind data
if os.path.exists('wind_data_historic/schleswig_holstein'):
    print('Data have already been extracted. Aborting.')
else:
    for file in os.listdir('wind_data_historic/historical'):
        if not file.endswith('.zip'):
            continue
        
        weather_station_id = file.split('_')[2]
        
        if not (weather_station_id in schleswig_holstein_weather_station_ids):
            continue
        
        with zipfile.ZipFile(f'wind_data_historic/historical/{file}', 'r') as zip_ref:
            zip_ref.extractall(f'wind_data_historic/schleswig_holstein')

Before continuing with the pre-processing and feature engineering, I will take a detour and look at the types of wind turbines in the data and their cut-in and cut-out speeds (ie. operational limits wrt wind speed).